# Project work, part 4
Sara Hørte

### Links

GitHub repo link: 
https://github.com/sarahorte/ind320project.git

Streamlit app link: 
https://ind320project.streamlit.app/

### Bonus exercise: 
Snow drift

### Log

Hente data 2021-2024. slette det som er i mongo fra før for å ha nok plass. gjør det først. se installation hvs jeg sliter.
også energy consumption.
Bytt Java 17 hvis problemer!

oppdatere plottene til plotly. tror spectrogram er eneste som mangler det.

map osv
- use choroplethmap. se pladlet
- session 1 17:00 10.nov


snow
-mulig wind drift ikke er dynamisk, og derfor ikke plotly
tror snow er den enkleste oppgaven. 



organisere sidene.


### AI usage
AI was used extensively throughout this project, primarily through ChatGPT and GitHub Copilot in VS Code. I leveraged AI to generate code suggestions, adapt examples from lectures, and refine solutions for specific tasks. For instance, when implementing the Local Outlier Factor method, I combined code snippets from lectures with AI-generated suggestions, then iteratively adjusted and tested the code until it functioned correctly.

I also used AI to interpret error messages and propose potential fixes. By testing different solutions suggested by ChatGPT, I was able to identify the most effective approach. Additionally, AI assisted in generating initial visualizations, which I later fine-tuned to achieve the desired appearance. For parameter selection in the various analysis functions, ChatGPT provided guidance, which was helpful as a starting point, though understanding the underlying concepts is essential for making meaningful choices.

The interactive nature of the Streamlit app made it easy to experiment with different parameters, and AI guidance accelerated this process. Beyond coding, I also used ChatGPT to refine and improve the wording of both the project log and the AI usage section itself, ensuring clarity and readability.

Overall, AI proved to be a valuable tool for both code development and documentation, complementing my own understanding and enabling more efficient problem-solving.

### Elhub data

In [10]:
# =============================
# FETCH DATA FROM ELHUB API (2021-2024) – FASTER
# =============================
import requests
import pandas as pd
from datetime import datetime, timedelta

# --- API SETTINGS ---
BASE_URL = "https://api.elhub.no/energy-data/v0/price-areas"

# datasets
DATASET_PROD = "PRODUCTION_PER_GROUP_MBA_HOUR"
DATASET_CONS = "CONSUMPTION_PER_GROUP_MBA_HOUR"

# --- DATE RANGES ---
PROD_START = datetime(2022, 1, 1)
PROD_END   = datetime(2024, 12, 31)

CONS_START = datetime(2021, 1, 1)
CONS_END   = datetime(2024, 12, 31)

# --- FUNCTION TO FORMAT DATES ---
def format_date(dt_obj):
    """Formats datetime for Elhub API (+02:00 offset)."""
    return dt_obj.strftime("%Y-%m-%dT%H:%M:%S%%2B02:00")  # +02:00 encoded

HEADERS = {"Accept": "application/json"}

# -----------------------
# Fetch data for a given month
# -----------------------
def fetch_month(dataset, year, month):
    start = datetime(year, month, 1)
    next_month = (start + timedelta(days=32)).replace(day=1)
    end = next_month - timedelta(seconds=1)

    url = f"{BASE_URL}?dataset={dataset}&startDate={format_date(start)}&endDate={format_date(end)}"
    resp = requests.get(url, headers=HEADERS, timeout=60)
    if resp.status_code != 200:
        print(f"❌ Error {resp.status_code} for {start.date()} → {end.date()}")
        return pd.DataFrame()

    data = resp.json().get("data", [])
    records = []
    for entry in data:
        attrs = entry.get("attributes", {})
        if dataset == DATASET_PROD:
            key = "productionPerGroupMbaHour"
            group_field = "productionGroup"
        else:
            key = "consumptionPerGroupMbaHour"
            group_field = "consumptionGroup"
        recs = attrs.get(key, [])
        recs = [r for r in recs if r.get(group_field) != "*" and r.get(group_field) is not None]
        for r in recs:
            records.append({
                "priceArea": r.get("priceArea"),
                "groupName": r.get(group_field),
                "startTime": r.get("startTime"),
                "endTime": r.get("endTime"),
                "quantityKwh": r.get("quantityKwh")
            })
    if records:
        df = pd.DataFrame(records)
        df['startTime'] = pd.to_datetime(df['startTime'], utc=True, errors='coerce')
        df['endTime'] = pd.to_datetime(df['endTime'], utc=True, errors='coerce')
        df['quantityKwh'] = pd.to_numeric(df['quantityKwh'], errors='coerce')
        df = df[['priceArea', 'groupName', 'startTime', 'endTime', 'quantityKwh']]
        df.sort_values('startTime', inplace=True)
        df.set_index('startTime', inplace=True)
        return df
    return pd.DataFrame()

# -----------------------
# Fetch all months in a range
# -----------------------
def fetch_range_monthly(dataset, start_dt, end_dt):
    all_dfs = []
    cur = start_dt
    while cur <= end_dt:
        df_month = fetch_month(dataset, cur.year, cur.month)
        if not df_month.empty:
            all_dfs.append(df_month)
        # move to next month
        next_month = (cur + timedelta(days=32)).replace(day=1)
        cur = next_month
    return pd.concat(all_dfs) if all_dfs else pd.DataFrame()

# -----------------------
# Run ingestion
# -----------------------
# Production 2022–2024
df_prod = fetch_range_monthly(DATASET_PROD, PROD_START, PROD_END)
print("Production data shape:", df_prod.shape)
print(df_prod.head())

# Consumption 2021–2024
df_cons = fetch_range_monthly(DATASET_CONS, CONS_START, CONS_END)
print("Consumption data shape:", df_cons.shape)
print(df_cons.head())


Production data shape: (657600, 4)
                          priceArea groupName                   endTime  \
startTime                                                                 
2021-12-31 23:00:00+00:00       NO1     hydro 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO4      wind 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO2     other 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO4   thermal 2022-01-01 00:00:00+00:00   
2021-12-31 23:00:00+00:00       NO1   thermal 2022-01-01 00:00:00+00:00   

                           quantityKwh  
startTime                               
2021-12-31 23:00:00+00:00   1291422.40  
2021-12-31 23:00:00+00:00    320912.40  
2021-12-31 23:00:00+00:00         0.20  
2021-12-31 23:00:00+00:00     38372.56  
2021-12-31 23:00:00+00:00     30049.92  
Consumption data shape: (876600, 4)
                          priceArea  groupName                   endTime  \
startTime                               

In [11]:
# ============================
# FIX DATAFRAMES FOR SPARK / CASSANDRA
# ============================

# 1️⃣ Reset index so 'startTime' becomes a column
df_prod = df_prod.reset_index()   # 'startTime' moves from index → column
df_cons = df_cons.reset_index()

# 2️⃣ Strip any extra whitespace from column names
df_prod.columns = df_prod.columns.str.strip()
df_cons.columns = df_cons.columns.str.strip()

# 3️⃣ Ensure uniform group column name
# (In your data, it's already 'groupName', but this covers any variations)
if 'productionGroup' in df_prod.columns:
    df_prod = df_prod.rename(columns={"productionGroup": "groupName"})
if 'consumptionGroup' in df_cons.columns:
    df_cons = df_cons.rename(columns={"consumptionGroup": "groupName"})

# 4️⃣ Ensure proper types
df_prod['startTime'] = pd.to_datetime(df_prod['startTime'], utc=True, errors='coerce')
df_prod['quantityKwh'] = pd.to_numeric(df_prod['quantityKwh'], errors='coerce')

df_cons['startTime'] = pd.to_datetime(df_cons['startTime'], utc=True, errors='coerce')
df_cons['quantityKwh'] = pd.to_numeric(df_cons['quantityKwh'], errors='coerce')

# 5️⃣ Drop rows with missing critical values
df_prod = df_prod.dropna(subset=['startTime','priceArea','groupName','quantityKwh'])
df_cons = df_cons.dropna(subset=['startTime','priceArea','groupName','quantityKwh'])

# 6️⃣ Optional: drop 'endTime' if not needed in Cassandra
df_prod = df_prod.drop(columns=['endTime'], errors='ignore')
df_cons = df_cons.drop(columns=['endTime'], errors='ignore')

# ✅ Check that DataFrames are ready
print("Production DataFrame ready:", df_prod.shape)
print("Consumption DataFrame ready:", df_cons.shape)
print("Production group names:", df_prod['groupName'].unique())
print("Consumption group names:", df_cons['groupName'].unique())


Production DataFrame ready: (657600, 4)
Consumption DataFrame ready: (876600, 4)
Production group names: ['hydro' 'wind' 'other' 'thermal' 'solar']
Consumption group names: ['cabin' 'tertiary' 'household' 'secondary' 'primary']


In [12]:
from cassandra.cluster import Cluster

# Connect to local Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# --- Create keyspace ---
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS energy
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")

# --- Create table for production ---
session.execute("""
    CREATE TABLE IF NOT EXISTS energy.production_per_group (
        priceArea text,
        startTime timestamp,
        productionGroup text,
        quantityKwh double,
        PRIMARY KEY ((priceArea), startTime, productionGroup)
    );
""")

# --- Create table for consumption ---
session.execute("""
    CREATE TABLE IF NOT EXISTS energy.consumption_per_group (
        priceArea text,
        startTime timestamp,
        consumptionGroup text,
        quantityKwh double,
        PRIMARY KEY ((priceArea), startTime, consumptionGroup)
    );
""")

print("✅ Keyspace and tables for production and consumption are ready")


✅ Keyspace and tables for production and consumption are ready


In [13]:
# -------------------------------
# Ensure startTime is timezone-aware UTC
# -------------------------------

import pytz

# Production
if df_prod['startTime'].dt.tz is None:
    df_prod['startTime'] = df_prod['startTime'].dt.tz_localize('UTC')
else:
    df_prod['startTime'] = df_prod['startTime'].dt.tz_convert('UTC')

# Consumption
if df_cons['startTime'].dt.tz is None:
    df_cons['startTime'] = df_cons['startTime'].dt.tz_localize('UTC')
else:
    df_cons['startTime'] = df_cons['startTime'].dt.tz_convert('UTC')

print("✅ startTime timezone fixed to UTC for both DataFrames")


✅ startTime timezone fixed to UTC for both DataFrames


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType
import math

# --- CREATE SPARK SESSION ---
spark = SparkSession.builder \
    .appName("ElhubDataIngest") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .getOrCreate()

print("Spark version:", spark.version)

# --- DEFINE COMMON SCHEMA ---
schema = StructType([
    StructField("priceArea", StringType(), True),
    StructField("groupName", StringType(), True),
    StructField("startTime", TimestampType(), True),
    StructField("quantityKwh", DoubleType(), True)
])

# --- OPTIMIZED FUNCTION TO WRITE IN CHUNKS ---
def write_to_cassandra_fast(df_pandas, dataset_type="production", chunk_size=100_000):
    if df_pandas is None or df_pandas.empty:
        print(f"No data to write for {dataset_type}")
        return

    # Drop any leftover index columns
    df_pandas = df_pandas.drop(columns=['index', 'level_0'], errors='ignore')

    total_rows = len(df_pandas)
    n_chunks = math.ceil(total_rows / chunk_size)
    print(f"Writing {total_rows} rows in {n_chunks} chunks for {dataset_type}...")

    for i in range(n_chunks):
        start_idx = i * chunk_size
        end_idx = min((i+1) * chunk_size, total_rows)
        df_chunk = df_pandas.iloc[start_idx:end_idx]

        # Make sure the columns match schema order exactly
        df_chunk_spark = df_chunk[['priceArea', 'groupName', 'startTime', 'quantityKwh']]

        # Convert to Spark DataFrame
        spark_df = spark.createDataFrame(df_chunk_spark, schema=schema)

        # Rename columns for Cassandra
        if dataset_type == "production":
            spark_df_cassandra = spark_df.selectExpr(
                "priceArea as pricearea",
                "startTime as starttime",
                "groupName as productiongroup",
                "quantityKwh as quantitykwh"
            )
            table = "production_per_group"
        else:
            spark_df_cassandra = spark_df.selectExpr(
                "priceArea as pricearea",
                "startTime as starttime",
                "groupName as consumptiongroup",
                "quantityKwh as quantitykwh"
            )
            table = "consumption_per_group"

        # Reduce Spark partitions to 1 per chunk for faster small writes
        spark_df_cassandra.coalesce(1).write \
            .format("org.apache.spark.sql.cassandra") \
            .options(keyspace="energy", table=table) \
            .mode("append") \
            .save()

        print(f"✅ Chunk {i+1}/{n_chunks} written ({len(df_chunk)} rows)")

# --- WRITE BOTH DATASETS ---
write_to_cassandra_fast(df_prod, "production")
write_to_cassandra_fast(df_cons, "consumption")


Spark version: 3.5.1
Writing 657600 rows in 7 chunks for production...


25/11/11 11:50:15 WARN TaskSetManager: Stage 0 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 1/7 written (100000 rows)


25/11/11 11:50:18 WARN TaskSetManager: Stage 1 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 2/7 written (100000 rows)


...

✅ Chunk 7/9 written (100000 rows)


25/11/11 11:50:38 WARN TaskSetManager: Stage 14 contains a task of very large size (3686 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 8/9 written (100000 rows)


25/11/11 11:50:39 WARN TaskSetManager: Stage 15 contains a task of very large size (2826 KiB). The maximum recommended task size is 1000 KiB.


✅ Chunk 9/9 written (76600 rows)


In [16]:
# ==========================================
# 3️⃣ READ DATA BACK FROM CASSANDRA
# ==========================================
def read_from_cassandra(table_name):
    """Reads a table from the 'energy' keyspace into a Spark DataFrame."""
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace="energy", table=table_name) \
        .load()
    return df

# --- Read production table ---
cass_prod_df = read_from_cassandra("production_per_group")
print("✅ Production table preview:")
cass_prod_df.show(5, truncate=False)

prod_count = cass_prod_df.count()
print(f"Total rows in production_per_group: {prod_count}")

# --- Read consumption table ---
cass_cons_df = read_from_cassandra("consumption_per_group")
print("\n✅ Consumption table preview:")
cass_cons_df.show(5, truncate=False)

cons_count = cass_cons_df.count()
print(f"Total rows in consumption_per_group: {cons_count}")

# --- Optional: basic sanity check on groups ---
print("\nProduction groups in Cassandra:", cass_prod_df.select("productiongroup").distinct().collect())
print("Consumption groups in Cassandra:", cass_cons_df.select("consumptiongroup").distinct().collect())


✅ Production table preview:
+---------+-------------------+---------------+-----------+
|pricearea|starttime          |productiongroup|quantitykwh|
+---------+-------------------+---------------+-----------+
|NO4      |2021-01-01 00:00:00|hydro          |3740830.0  |
|NO4      |2021-01-01 00:00:00|other          |0.161      |
|NO4      |2021-01-01 00:00:00|solar          |0.0        |
|NO4      |2021-01-01 00:00:00|thermal        |21349.0    |
|NO4      |2021-01-01 00:00:00|wind           |381065.0   |
+---------+-------------------+---------------+-----------+
only showing top 5 rows



Total rows in production_per_group: 872853

✅ Consumption table preview:
+---------+-------------------+----------------+-----------+
|pricearea|starttime          |consumptiongroup|quantitykwh|
+---------+-------------------+----------------+-----------+
|NO4      |2021-01-01 00:00:00|cabin           |56058.316  |
|NO4      |2021-01-01 00:00:00|household       |675104.9   |
|NO4      |2021-01-01 00:00:00|primary         |51439.465  |
|NO4      |2021-01-01 00:00:00|secondary       |848937.56  |
|NO4      |2021-01-01 00:00:00|tertiary        |366046.0   |
+---------+-------------------+----------------+-----------+
only showing top 5 rows

Total rows in consumption_per_group: 876500



Production groups in Cassandra: [Row(productiongroup='solar'), Row(productiongroup='other'), Row(productiongroup='thermal'), Row(productiongroup='hydro'), Row(productiongroup='wind')]
Consumption groups in Cassandra: [Row(consumptiongroup='tertiary'), Row(consumptiongroup='secondary'), Row(consumptiongroup='household'), Row(consumptiongroup='cabin'), Row(consumptiongroup='primary')]


### Elhub data

In [2]:
# Fetch and store Elhub production & consumption hourly data (2021-2024)
# Adapted from your earlier notebook
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from dateutil import tz
import pytz
import math
from tqdm import tqdm

# Cassandra / Spark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType

# MongoDB
from pymongo import MongoClient, ASCENDING
from pymongo.errors import BulkWriteError, DuplicateKeyError

# -----------------------
# Configuration
# -----------------------
BASE_URL = "https://api.elhub.no/energy-data/v0/price-areas"
# datasets
DATASET_PROD = "PRODUCTION_PER_GROUP_MBA_HOUR"
DATASET_CONS = "CONSUMPTION_PER_GROUP_MBA_HOUR"

# Date ranges
PROD_START = datetime(2022, 1, 1)
PROD_END   = datetime(2024, 12, 31, 23, 59, 59)

CONS_START = datetime(2021, 1, 1)
CONS_END   = datetime(2024, 12, 31, 23, 59, 59)

# API timezone formatting: your previous code used +02:00 encoded as %2B02:00
def format_date(dt_obj):
    """Format datetime in the Elhub expected format with +02:00 percent-encoded.
    We send naive datetimes in local CET/CEST? To be consistent with your earlier code,
    we format as YYYY-MM-DDTHH:MM:SS%2B02:00
    """
    return dt_obj.strftime("%Y-%m-%dT%H:%M:%S%%2B02:00")  # +02:00 encoded

HEADERS = {
    "Accept": "application/json"
}

# Networking / retry config
MAX_RETRIES = 5
SLEEP_BETWEEN_REQUESTS = 0.6  # gentle
BACKOFF_FACTOR = 2.0

# Local DB hosts
CASSANDRA_HOST = "127.0.0.1"
MONGO_URI = "mongodb://127.0.0.1:27017"
MONGO_DBNAME = "energy"

# -----------------------
# Helper: request with retry
# -----------------------
def get_with_retry(url, max_retries=MAX_RETRIES):
    wait = 1.0
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(url, headers=HEADERS, timeout=60)
            if r.status_code == 200:
                return r.json()
            else:
                print(f"Warning: status {r.status_code} for {url}")
        except requests.RequestException as e:
            print(f"RequestException: {e} (attempt {attempt}/{max_retries})")
        if attempt < max_retries:
            time.sleep(wait)
            wait *= BACKOFF_FACTOR
    raise RuntimeError(f"Failed to fetch {url} after {max_retries} attempts")

# -----------------------
# Extract records from response
# -----------------------
def extract_records_from_response(response_json, dataset):
    """Return list of dict records with fields: priceArea, group, startTime, quantityKwh"""
    data = response_json.get("data", [])
    all_recs = []
    if dataset == DATASET_PROD:
        key = "productionPerGroupMbaHour"
        group_field = "productionGroup"
    elif dataset == DATASET_CONS:
        key = "consumptionPerGroupMbaHour"
        group_field = "consumptionGroup"
    else:
        raise ValueError("Unknown dataset")

    for entry in data:
        attrs = entry.get("attributes", {})
        recs = attrs.get(key, [])
        # Filter placeholder groups
        for r in recs:
            if r.get(group_field) == "*" or r.get(group_field) is None:
                continue
            # Build normalized record
            rec = {
                "priceArea": r.get("priceArea"),
                "group": r.get(group_field),
                "startTime": r.get("startTime"),
                "endTime": r.get("endTime"),
                "quantityKwh": r.get("quantityKwh")
            }
            all_recs.append(rec)
    return all_recs

# -----------------------
# Fetch in daily chunks (safer than monthly)
# -----------------------
def fetch_range_daily(dataset, start_dt, end_dt, sleep=SLEEP_BETWEEN_REQUESTS):
    """Yield DataFrame chunks for each day in [start_dt, end_dt]."""
    records = []
    cur = start_dt
    # ensure start at midnight
    cur = cur.replace(hour=0, minute=0, second=0, microsecond=0)
    end_limit = end_dt
    while cur <= end_limit:
        day_start = cur
        day_end = cur + timedelta(days=1) - timedelta(seconds=1)
        start_str = format_date(day_start)
        end_str = format_date(day_end)
        url = f"{BASE_URL}?dataset={dataset}&startDate={start_str}&endDate={end_str}"
        # print progress per day
        try:
            resp = get_with_retry(url)
        except Exception as e:
            print(f"ERROR fetching {day_start.date()}: {e}")
            cur += timedelta(days=1)
            time.sleep(sleep)
            continue

        recs = extract_records_from_response(resp, dataset)
        if recs:
            df = pd.DataFrame(recs)
            # parse times; Elhub uses timezone suffix; parse with pandas
            df['startTime'] = pd.to_datetime(df['startTime'], utc=True, errors='coerce')
            df['endTime'] = pd.to_datetime(df['endTime'], utc=True, errors='coerce')
            df['quantityKwh'] = pd.to_numeric(df['quantityKwh'], errors='coerce')
            df = df.rename(columns={"group": "groupName"})
            # normalize column names
            df = df[['priceArea', 'groupName', 'startTime', 'endTime', 'quantityKwh']]
            yield df
        # be kind to API
        time.sleep(sleep)
        cur += timedelta(days=1)

# -----------------------
# Spark session (for Cassandra writes)
# -----------------------
spark = SparkSession.builder \
    .appName("ElhubDataIngest") \
    .config("spark.cassandra.connection.host", CASSANDRA_HOST) \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .getOrCreate()

print("Spark version:", spark.version)

# Ensure Cassandra keyspace and tables exist
from cassandra.cluster import Cluster
cluster = Cluster([CASSANDRA_HOST])
cass_session = cluster.connect()
cass_session.execute("""
    CREATE KEYSPACE IF NOT EXISTS energy
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")

# Production table
cass_session.execute("""
    CREATE TABLE IF NOT EXISTS energy.production_per_group (
        priceArea text,
        startTime timestamp,
        productionGroup text,
        quantityKwh double,
        PRIMARY KEY ((priceArea), startTime, productionGroup)
    );
""")

# Consumption table
cass_session.execute("""
    CREATE TABLE IF NOT EXISTS energy.consumption_per_group (
        priceArea text,
        startTime timestamp,
        consumptionGroup text,
        quantityKwh double,
        PRIMARY KEY ((priceArea), startTime, consumptionGroup)
    );
""")

print("✅ Cassandra keyspace and tables ready")



Spark version: 3.5.1
✅ Cassandra keyspace and tables ready


25/11/11 10:45:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# -----------------------
# MongoDB setup
# -----------------------
mongo_client = MongoClient(MONGO_URI)
mdb = mongo_client[MONGO_DBNAME]
prod_col = mdb["production_per_group"]
cons_col = mdb["consumption_per_group"]


# create unique indexes to prevent duplicates on (priceArea, startTime, groupName)
prod_col.create_index(
    [("priceArea", ASCENDING), ("startTime", ASCENDING), ("groupName", ASCENDING)],
    unique=True, name="prod_unique_idx"
)
cons_col.create_index(
    [("priceArea", ASCENDING), ("startTime", ASCENDING), ("groupName", ASCENDING)],
    unique=True, name="cons_unique_idx"
)
print("✅ MongoDB indexes created/ensured")

# -----------------------
# Common schema for Spark DF creation
# -----------------------
schema = StructType([
    StructField("priceArea", StringType(), True),
    StructField("groupName", StringType(), True),
    StructField("startTime", TimestampType(), True),
    StructField("endTime", TimestampType(), True),
    StructField("quantityKwh", DoubleType(), True)
])

# Helper to write pandas df to Cassandra using Spark
def write_to_cassandra(df_pandas, dataset_type="production"):
    if df_pandas is None or df_pandas.empty:
        return
    # Convert pandas -> spark
    spark_df = spark.createDataFrame(df_pandas.astype(object), schema=schema)
    # rename columns per Cassandra table
    if dataset_type == "production":
        spark_df_cassandra = spark_df.selectExpr(
            "priceArea as pricearea",
            "startTime as starttime",
            "groupName as productiongroup",
            "quantityKwh as quantitykwh"
        )
        keyspace = "energy"
        table = "production_per_group"
    else:
        spark_df_cassandra = spark_df.selectExpr(
            "priceArea as pricearea",
            "startTime as starttime",
            "groupName as consumptiongroup",
            "quantityKwh as quantitykwh"
        )
        keyspace = "energy"
        table = "consumption_per_group"

    # write in append mode
    spark_df_cassandra.write \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace=keyspace, table=table) \
        .mode("append") \
        .save()

# Helper to write to MongoDB (bulk upsert-ish by ignoring duplicates)
def write_to_mongo(df_pandas, collection):
    if df_pandas is None or df_pandas.empty:
        return
    # prepare bulk inserts (Mongo will reject duplicates due to unique index)
    docs = []
    for _, r in df_pandas.iterrows():
        doc = {
            "priceArea": r['priceArea'],
            "groupName": r['groupName'],
            "startTime": pd.to_datetime(r['startTime']).to_pydatetime(),
            "endTime": pd.to_datetime(r['endTime']).to_pydatetime(),
            "quantityKwh": None if pd.isna(r['quantityKwh']) else float(r['quantityKwh'])
        }
        docs.append(doc)
    if not docs:
        return
    try:
        collection.insert_many(docs, ordered=False)
    except BulkWriteError as bwe:
        # most likely duplicate key errors; ignore and continue
        print("Mongo BulkWriteError (likely duplicates). Inserted some documents; duplicates skipped.")
    except DuplicateKeyError:
        print("Mongo DuplicateKeyError (skipped)")

# -----------------------
# Ingest loops
# -----------------------
def ingest_dataset(dataset, start_dt, end_dt, target_mongo_collection, dataset_type):
    total_rows = 0
    day_count = (end_dt.date() - start_dt.date()).days + 1
    print(f"Starting ingest for {dataset} from {start_dt.date()} to {end_dt.date()} ({day_count} days)")
    for df_day in tqdm(fetch_range_daily(dataset, start_dt, end_dt), desc=f"Ingesting {dataset_type}"):
        # df_day has columns: priceArea, groupName, startTime, endTime, quantityKwh
        if df_day.empty:
            continue
        # drop rows with null key fields
        df_day = df_day.dropna(subset=['priceArea', 'groupName', 'startTime'])
        if df_day.empty:
            continue

        # write to MongoDB
        write_to_mongo(df_day, target_mongo_collection)

        # write to Cassandra via Spark
        # Note: our Cassandra table doesn't include endTime: we create Spark df with endTime in schema but only map the needed columns
        write_to_cassandra(df_day, dataset_type)

        total_rows += len(df_day)
    print(f"Finished ingest for {dataset_type}. Total rows processed: {total_rows}")

# Run production ingest (2022-2024)
ingest_dataset(DATASET_PROD, PROD_START, PROD_END, prod_col, dataset_type="production")

# Run consumption ingest (2021-2024)
ingest_dataset(DATASET_CONS, CONS_START, CONS_END, cons_col, dataset_type="consumption")

# -----------------------
# Basic verification prints
# -----------------------
print("Mongo counts (approx):")
print("production:", prod_col.count_documents({}))
print("consumption:", cons_col.count_documents({}))

# Spark read-back sample (production)
print("Sample rows from Cassandra production_per_group:")
cass_prod = spark.read.format("org.apache.spark.sql.cassandra").options(keyspace="energy", table="production_per_group").load()
cass_prod.show(5)

print("Done.")


ServerSelectionTimeoutError: 127.0.0.1:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 691305deffb406b1a4413861, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('127.0.0.1:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CheckJava").getOrCreate()
print("Java version (Spark):", spark.sparkContext._gateway.jvm.java.lang.System.getProperty("java.version"))


25/11/11 10:44:45 WARN Utils: Your hostname, Sara-sin-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.28.28.196 instead (on interface en0)
25/11/11 10:44:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/11 10:44:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Java version (Spark): 17.0.17


In [7]:
spark.stop()


In [1]:
import os
print(os.environ.get("JAVA_HOME"))


/Library/Java/JavaVirtualMachines/microsoft-11.jdk/Contents/Home
